In [97]:
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

In [98]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-s_vw27bf/kobert-tokenizer_a208798a9d38422dad0acb838035ae4a
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-s_vw27bf/kobert-tokenizer_a208798a9d38422dad0acb838035ae4a


In [99]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master 

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3paozwpq
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3paozwpq


In [100]:
!pip uninstall folium

Found existing installation: folium 0.2.1
Uninstalling folium-0.2.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/folium-0.2.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/folium/*
Proceed (y/n)? y
  Successfully uninstalled folium-0.2.1


In [101]:
!pip install folium==0.2.1

  Using cached folium-0.2.1-py3-none-any.whl


In [102]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [103]:
from kobert_tokenizer import KoBERTTokenizer

In [104]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_pytorch_kobert_model()
# print(bertModel, vocab_obj)

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [105]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

In [106]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla P100-PCIE-16GB


In [107]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Model1_data/감성대화/Training/감성대화말뭉치(최종데이터)_Training/gamsung_test_data.csv')
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164.0,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926.0,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,50882.0,청소년,여성,학업 및 진로,해당없음,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,31303.0,노년,남성,재정,만성질환 무,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,47200.0,노년,여성,재정,만성질환 유,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# data.isnull().sum()

In [110]:
# data1 = data.dropna()
# data1.isnull().sum()

In [111]:
data['감정_대분류'].unique()
data['감정_대분류'].value_counts()

불안     7328
분노     6920
슬픔     6915
기쁨     6730
상처     6624
당황     6360
불안        1
기쁨        1
Name: 감정_대분류, dtype: int64

In [112]:
data1 = data[['사람문장1', '감정_대분류']]
data1

,사람문장1,감정_대분류
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,불안
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,당황
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기쁨
...,...,...
69213,NaN,NaN
69214,NaN,NaN
69215,NaN,NaN
69216,NaN,NaN


In [113]:
data1.loc[(data['감정_대분류']=='기쁨'), '감정_대분류'] = 0
data1.loc[(data['감정_대분류']=='분노'), '감정_대분류'] = 1
data1.loc[(data['감정_대분류']=='슬픔'), '감정_대분류'] = 2
data1.loc[(data['감정_대분류']=='불안'), '감정_대분류'] = 3
data1.loc[(data['감정_대분류']=='상처'), '감정_대분류'] = 4
data1.loc[(data['감정_대분류']=='당황'), '감정_대분류'] = 5
data1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,사람문장1,감정_대분류
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,3
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,5
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0
...,...,...
69213,NaN,NaN
69214,NaN,NaN
69215,NaN,NaN
69216,NaN,NaN


In [114]:
data1.loc[(data['감정_대분류']=='기쁨 '), '감정_대분류'] = 0
data1.loc[(data['감정_대분류']=='불안 '), '감정_대분류'] = 3
# data.loc[(data['감정_대분류']=='분노 '), '감정_대분류'] = 1
# data.loc[(data['감정_대분류']=='슬픔 '), '감정_대분류'] = 2
# data.loc[(data['감정_대분류']=='상처 '), '감정_대분류'] = 4
# data.loc[(data['감정_대분류']=='당황 '), '감정_대분류'] = 5
data1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,사람문장1,감정_대분류
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,3
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,5
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0
...,...,...
69213,NaN,NaN
69214,NaN,NaN
69215,NaN,NaN
69216,NaN,NaN


In [115]:
data1.isnull().sum()

사람문장1     28339
감정_대분류    28339
dtype: int64

In [116]:
# 여기서 결측값 처리하기
data2 = data1.dropna()
data2.isnull().sum()

사람문장1     0
감정_대분류    0
dtype: int64

In [117]:
len(data2)

40879

In [118]:
data2

,사람문장1,감정_대분류
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,3
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,5
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0
...,...,...
40874,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,3
40875,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어.,0
40876,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.,2
40877,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야.,0


In [119]:
data2['감정_대분류'].unique()
data2['감정_대분류'].value_counts()

3    7329
1    6920
2    6915
0    6731
4    6624
5    6360
Name: 감정_대분류, dtype: int64

In [120]:
# data2.loc[(data['감정_대분류']=='기쁨 '), '감정_대분류'] = 0
# data2.loc[(data['감정_대분류']=='분노 '), '감정_대분류'] = 1
# data2.loc[(data['감정_대분류']=='슬픔 '), '감정_대분류'] = 2
# data2.loc[(data['감정_대분류']=='불안 '), '감정_대분류'] = 3
# data2.loc[(data['감정_대분류']=='상처 '), '감정_대분류'] = 4
# data2.loc[(data['감정_대분류']=='당황 '), '감정_대분류'] = 5
# data2

In [121]:
data_list = []
for ques, label in zip(data2['사람문장1'], data2['감정_대분류']):
  test_data = []
  test_data.append(ques)
  test_data.append(str(label))

  data_list.append(test_data)
data_list
len(data_list)

40879

In [122]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [156]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [124]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)
len(dataset_train)
# len(dataset_test)

32703

In [125]:
# dataset_train[4889][1] = dataset_train[4889][1].rstrip()
# dataset_train[4889]

In [126]:
# dataset_train.loc[(data['감정_대분류']=='불안'), '감정_대분류'] = 3
# dataset_train

In [127]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [128]:
# for li in dataset_train:

#   if isdemical(li[1]):

In [129]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [149]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ## 분류하고 싶은 감정 수 (긍정, 부정으로 할거면 2로 수정) ##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out), pooler # 64 6 

In [151]:
#BERT 모델 불러오기
# dr_rate 를 높여도 정확도가 높아지진 않음. 
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [157]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out, pooler = model(token_ids, valid_length, segment_ids)
        if batch_id == 1:
          break
      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/511 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/511 [00:00<?, ?it/s]

  0%|          | 0/511 [00:00<?, ?it/s]

  0%|          | 0/511 [00:00<?, ?it/s]

  0%|          | 0/511 [00:00<?, ?it/s]

In [159]:
out.shape

torch.Size([64, 6])

In [158]:
pooler.shape

torch.Size([64, 768])

In [142]:
out[0]

tensor([-2.9998, -1.0629,  0.8907, -1.4038,  3.1487,  1.4626], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [132]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out, pooler = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/511 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.9186931848526 train acc 0.140625
epoch 1 batch id 201 loss 1.3589048385620117 train acc 0.3293687810945274
epoch 1 batch id 401 loss 1.0406770706176758 train acc 0.4600218204488778
epoch 1 train acc 0.49347975491566487


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6451416015625


  0%|          | 0/511 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.6256652474403381 train acc 0.78125
epoch 5 batch id 201 loss 0.6367164850234985 train acc 0.784981343283582
epoch 5 batch id 401 loss 0.509421169757843 train acc 0.8015508104738155
epoch 5 train acc 0.8053109371602523


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 5 test acc 0.646240234375


In [133]:
from google.colab import drive
drive.mount('/content/drive')
test_data = pd.read_csv('/content/drive/MyDrive/Model1_data/감성대화/Validation/감성대화말뭉치(최종데이터)_Validation.csv')
test_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,23093,청년,여성,"진로, 취업, 직장",해당없음,분노,툴툴대는,요즘 부모님과 많이 부딪혀.,어떤 일로 부딪히시나요?,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...,부모님과의 충돌로 힘드시겠군요.,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.,자신이 하고 싶은 일을 목표로 하기로 하셨군요.,NaN,NaN
1,32848,청소년,남성,가족관계,해당없음,슬픔,비통한,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,어머니께서 집을 나가셨다니 정말 슬프셨겠어요. 좀 더 자세히 얘기해주시겠어요?,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...,정말 많이 답답하셨겠어요. 어떻게 하는 게 기분이 좀 풀릴까요?,일단 좀 자야겠어.,일단 휴식을 가질 생각이시군요.,NaN,NaN
2,35590,청소년,남성,학교폭력/따돌림,해당없음,불안,조심스러운,학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어.,용기 있는 행동을 하셨네요. 마음이 어떠셨나요?,그런데 말을 하고 난 후 나도 괴롭힐까 봐 불안했어.,그랬군요. 지금 기분에서 벗어나기 위해 할 수 있는 일에는 무엇이 있을까요?,나는 옳은 일을 한 거라고 생각해. 나까지 괴롭힌다면 신고할 거야.,지금의 상황이 나의 방식으로써 잘 해결될 수 있기를 바라요.,NaN,NaN
3,169,청년,남성,"진로, 취업, 직장",해당없음,당황,죄책감의,이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야.,팀장님이 부탁하신 업무를 잘못 처리하셔서 많이 당황하시고 놀라셨겠어요.,곧 있으면 인턴에서 정규직 전환 평가가 있는데 안 좋을 것 같아.,마음이 많이 안 좋으시겠어요. 지금 상황을 타개할 수 있는 가장 좋은 방안이 있을까요?,우선 잘못된 부분을 솔직하게 말씀드리고 수정하기 위해서 최선을 다해봐야겠어.,잘못된 부분을 잘 수정해서 좋은 결과가 있었으면 좋겠어요.,NaN,NaN
4,38435,중년,여성,"재정, 은퇴, 노후준비",해당없음,분노,노여워하는,남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어.,많이 화가 나시겠어요.,어떻게 그럴 수가 있지? 너무 화가 나.,어떻게 하면 지금의 감정을 추스를 수 있을까요?,나도 변호사를 고용해서 한번 얘기해보려고 해.,자녀분들이랑 함께 하며 슬픔을 극복하시길 바라요.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5125,37859,중년,남성,"재정, 은퇴, 노후준비",해당없음,기쁨,느긋,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,여유롭게 생활할 수 있어 좋으시겠어요.,그래서 마음의 안정을 가질 수 있었던 것 같아.,그렇군요. 계속해서 여유로우려면 어떡해야 좋을까요?,돈을 함부로 쓰지 말고 지금 현재에 만족하면서 지내야지.,다른 병원에 가보려고 하시는군요.,NaN,NaN
5126,40171,노년,여성,"건강, 죽음",만성질환 유,불안,회의적인,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,치료가 완료되지 않은 게 아닐까 싶으셔서 염려하고 계시는군요.,이대로 혹시 더 심해져서 죽게 되는 건 아닐까?,많이 불안하시군요. 불안함을 줄일 수 있는 방법이 있을까요?,의사 선생님에게 악화될 확률은 얼마나 되는지 정확히 물어봐야겠어.,정확히 진단받고 나면 염려를 줄이는데 도움이 될 거예요.,NaN,NaN
5127,13717,청년,남성,"연애, 결혼, 출산",해당없음,분노,좌절한,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,소개팅에서 만나는 사람과 계속 잘 안 풀려서 많이 좌절하신 것 같아요.,내가 게을러서 다른 방법을 아직 생각하진 못했어. 댄스 동호회는 좀 괜찮은 것 같은...,댄스 동호회에 가입해서 사람들을 만나려고 하시는군요.,NaN,NaN,NaN,NaN
5128,33784,노년,남성,재정,만성질환 유,분노,안달하는,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해.,은행 대출이 안 돼서 마음이 괴로우시군요.,당장 내일 밥은 어떡하고 내 병원비는 어쩌지.,이 상황을 해결하기 위해서는 어떻게 할 수 있을까요?,어떻게 해야 할 지 모르겠어. 눈 앞이 캄캄해.,지금 이 기분을 나눌 수 있는 사람이 주위에 있으면 좋겠어요.,NaN,NaN


In [134]:
contents = test_data['사람문장1']
test_list = []
for i in range(len(contents[0:100])):
  test_list.append(contents[i])
test_list

['요즘 부모님과 많이 부딪혀.',
 '엄마가 결국 집을 나갔어. 너무 너무 슬퍼.',
 '학교에서 한 친구를 괴롭히는 무리에게 그만하라고 했어.',
 '이번에 팀장님이 간단한 조사 업무를 부탁하셨는데 내가 잘못 처리했어. 너무 절망적이야.',
 '남편이 이혼할 때 위자료를 주지 않으려고 변호사를 고용했어.',
 '친구들과 노후에 대한 이야기를 하다 보니 서로 노후 자금 차이가 상당히 컸어.',
 '직장에서 모함을 받았어. 난 정말 억울해.',
 '요즘 딸에게 뭔가 물어보면 신경질부터 내는데 화가 나.',
 '언제까지 대출금을 갚으며 살아야 할까? 포기하고 싶어지고 너무 슬퍼.',
 '이 업무는 하루 만에 끝낼 수 있을 것 같아.',
 '나 대학에 쉽게 갈 수 있을 것 같아. 학원 선생님들 실력이 정말 믿을 만해!',
 '작년에 왔던 이직 제안을 거절한 게 후회돼.',
 '나이는 먹어 가는데 경제적인 여유가 없어서 큰일이네.',
 '이번에 약이 바뀌었는데 정말 구역질이 나.',
 '내 친구가 따돌림을 당하고 있어.',
 '엄마는 나를 걱정해서 충고를 했는데 나는 짜증만 냈어.',
 '심혈을 기울여 작성한 기획안이 통과되었어!',
 '친구들과 단톡방을 나와버렸어.',
 '잘나가는 친척 동생이랑 자꾸 나랑 비교당하면서 내가 위축이 돼.',
 '남자는 대체 왜 그러니? 너무 혐오스러워.',
 '학교폭력 가해자로 낙인찍혀서 우리 부모님까지 손가락질을 당했어.',
 '좋아하던 선배에게 어제 고백을 받았어!',
 '아들에게는 없었는데 아내에게는 가끔 당혹스러울 때가 있어.',
 '친구가 없어서 학교에서 밥을 혼자 먹는 게 너무 부끄러워.',
 '아내에게 양치질 좀 하라고 했는데 말을 또 안 들어. 어휴 화나.',
 '내 친구들은 공부를 잘 하는데 선생님이 온갖 편의를 누리게 해줘서 부담스러워.',
 '오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.',
 '지금 만나는 사람과 연애를 시작했는데 결혼까지는 생각이 안 들어서 고민이야.',
 '내가 

In [176]:
def predict(predict_sentence):
  # 재귀함수로 list 를 for 문으로 돌려서 함수 계속 호출

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        # print(out)
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            print("2222") # 4 # 638
            print(np.argmax(logits))
            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("분노가")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("불안은")
            elif np.argmax(logits) == 4:
                test_eval.append("상처가")
            elif np.argmax(logits) == 5:
                test_eval.append("당황이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
for test in test_list:
  print(test, ':')
  predict(test)

In [201]:

#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    # print(out.shape)
    print("\n")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


tensor([[ 0.1590,  0.1500,  0.2354, -0.4068, -0.0438, -0.1649]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용에서 슬픔이 느껴집니다.
tensor([[ 0.0086, -0.1806, -0.2954,  0.0429,  0.1799,  0.1264]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용에서 상처가 느껴집니다.
tensor([[-0.0707, -0.3385, -0.3490,  0.2367,  0.3394,  0.3511]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용에서 당황이 느껴집니다.
tensor([[-0.0881, -0.2779, -0.2533,  0.2438,  0.0892,  0.4256]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용에서 당황이 느껴집니다.
tensor([[-0.1634, -0.1863, -0.1911,  0.3344,  0.3768,  0.4848]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용에서 당황이 느껴집니다.
tensor([[ 0.2401,  0.0828,  0.3574, -0.4541, -0.1820, -0.1093]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용에서 슬픔이 느껴집니다.
tensor([[-0.1191, -0.1067, -0.3060,  0.1654,  0.2121,  0.3596]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
>> 입력하신 내용에서 당황이 느껴집니다.
tensor([[-0.1355,  0.5452, 

KeyboardInterrupt: ignored

In [ ]:
|